In [1]:
import sqlite3
import os
import sys
import time
import pandas as pd
import numpy as np
import preprocesser

import datetime
from dateutil.parser import parse

In [11]:
import talib

In [2]:
MARKET_KOSPI   = 0
MARKET_KOSDAQ  = 10

In [2]:
#chart_data = pd.read_csv('C:\\Users\\JUNKIM\\python\\02.stock\\gatherer_junkim\\000000.csv')
chart_data = pd.read_csv('C:\\Users\\312263\\Desktop\\python\\03_stock\\000000.csv')
chart_data.columns = ['date', 'open', 'high', 'low', 'close', 'volume']

종목코드열람

In [3]:
# SQLite DB 연결
conn = sqlite3.connect("t8430.db")
# Connection 으로부터 Cursor 생성
cur = conn.cursor()
# SQL 쿼리 실행
query = "select distinct 종목명, 종목코드 \
        from 종목코드 ;"
        #where 종목코드 = '%s'; " % '000020'
cur.execute(query)
# 데이타 Fetch
rows = cur.fetchall()
shcode_list = pd.DataFrame(rows, columns = ['종목명', '종목코드'])
# Connection 닫기
conn.close()

In [4]:
shcode_list.head()

,종목명,종목코드
0,동화약품,000020
1,KR모터스,000040
2,경방,000050
3,메리츠화재,000060
4,삼양홀딩스,000070


시세데이터 뽑기

In [19]:
def query_ohlcv(shcode):
    # SQLite DB 연결
    conn = sqlite3.connect("t1305.db")
    # Connection 으로부터 Cursor 생성
    cur = conn.cursor()
    # SQL 쿼리 실행
    query = "select 일자, 시가, 고가, 저가, 종가, 누적거래량 \
            from 종목별체결조회 \
            where 1=1 \
            and 종목코드 = '%s' \
            and 일주월구분 = '%s' ;" % (shcode,'3')
    cur.execute(query)
    # 데이타 Fetch
    query_result = cur.fetchall()
    fin_result = pd.DataFrame(query_result, columns = ['date', 'open', 'high', 'low', 'close', 'volume'])
    # Connection 닫기
    daily = fin_result.iloc[:100].sort_values(by = ['date']).reset_index(drop = True)
    weekly = fin_result.iloc[100:200].sort_values(by = ['date']).reset_index(drop = True)
    monthly = fin_result.iloc[200:].sort_values(by = ['date']).reset_index(drop = True)
    conn.close()
    return daily, weekly, monthly

In [20]:
daily_data, weekly_data, monthly_data = query_ohlcv(shcode_list['종목코드'].iloc[0])

In [5]:
df['date'] = df['date'].apply(lambda x: parse(x).strftime('%Y%m%d'))

In [6]:
df_new = pd.DataFrame([],columns = ['date', 'open', 'high', 'low', 'close', 'volume'])

In [10]:
#batch_size * steps_per_epoch =s 전체샘플수
lookback = 5
step = 1
delay = 0 #바로다음거 타겟_ cls 미사용
batch_size = 1


def generator_cls(data_x, data_y, lookback, delay, min_index, max_index,
              shuffle=False, batch_size=1, step=1):
    #끝점
    if max_index is None:
        max_index = len(data_x) - delay - 1  
    #시작점
    i = min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            #시작점부터 인덱스최대와 시작점+배치사이즈 사이의 모든 행을 리스트로가져옴
            rows = np.arange(i, min(i + batch_size, max_index))
            #시작점 + 총행의길이
            i += len(rows)

        samples = np.zeros((len(rows),         # 총개수
                           lookback // step,   # 하나입력데이터개수
                           data_x.shape[-1]))  # 열갯수
        
        targets = np.zeros((len(rows),         # 총개수
                           data_y.shape[-1]))  # 열개수
        
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)  #(시작점 -lookback)->최초시작점 부터 시작점까지 step수만큼
            samples[j] = data_x[indices]
            targets[j] = data_y[rows[j]-1][:]  #target data 변경
            
            
        yield samples, targets

In [11]:
df_new.head(10)

,date,open,high,low,close,volume
0,20171231,698.64,730.03,682.16,727.45,26063.61900
1,20180101,727.01,763.55,711.10,742.74,56434.92261
2,20180102,742.41,899.50,733.00,856.93,91952.80814
3,20180103,856.92,884.98,810.00,868.89,95818.07760
4,20180104,868.86,1009.72,859.84,1002.31,94631.18420
5,20180105,1006.78,1045.00,910.00,981.97,105250.84204
6,20180106,981.97,1005.00,930.00,992.46,82885.18727
7,20180107,992.45,1095.90,982.00,1082.61,62576.36663
8,20180108,1082.61,1239.18,951.00,970.11,121768.95634
9,20180109,970.12,1260.00,910.00,1202.01,141992.87571


In [13]:
df_new.shape

(575, 6)

In [12]:
df_new2 = df_new[['date', 'open', 'high', 'low', 'close', 'volume']].values
df_new2_target = df_new[['close']].values


In [14]:
gen = generator_cls(df_new2,df_new2_target, lookback, delay, min_index=0, max_index=575,
                                shuffle=False, batch_size=1, step=1)

In [33]:
a = next(gen)[0]
b = next(gen)[1]

In [34]:
pd.DataFrame(a[0],columns = ['date', 'open', 'high', 'low', 'close', 'volume'])

,date,open,high,low,close,volume
0,20180104.0,868.86,1009.72,859.84,1002.31,94631.18420
1,20180105.0,1006.78,1045.00,910.00,981.97,105250.84204
2,20180106.0,981.97,1005.00,930.00,992.46,82885.18727
3,20180107.0,992.45,1095.90,982.00,1082.61,62576.36663
4,20180108.0,1082.61,1239.18,951.00,970.11,121768.95634


In [32]:
b

array([[1082.61]])

In [35]:
c = next(gen)

In [38]:
c[1]

array([[1308.61]])

In [ ]:
pd.DataFrame(next(gen)[1][0],columns = ['date', 'open', 'high', 'low', 'close', 'volume'])

In [101]:
all([high_vol >= average_vol * 10, 
    high_vol * a[0][:,5][ind] >= 1000])

False

In [114]:
ratio_list

[1.0, 0.5, 0.33333333333333326, 0.25]

In [120]:
np.array(ratio_list).argmax()

0

In [ ]:
#>>> a = [1,2,3,4]
#>>> result = [num * 3 for num in a if num % 2 == 0]
#>>> print(result)
#[6, 12]

In [51]:
daily_data, weekly_data, monthly_data = query_ohlcv(shcode_list['종목코드'].iloc[1])

In [52]:
preprocesser.sma(daily_data)
preprocesser.sma(daily_data)
preprocesser.macd(daily_data)
daily_data.head(20)

,date,open,high,low,close,volume,sma5,sma10,sma20,sma60,macd,macd_signal
0,20190430,530,595,530,550,2154890,NaN,NaN,NaN,NaN,NaN,NaN
1,20190502,545,554,532,544,425141,NaN,NaN,NaN,NaN,NaN,NaN
2,20190503,544,560,538,557,391150,NaN,NaN,NaN,NaN,NaN,NaN
3,20190507,557,567,544,558,311506,NaN,NaN,NaN,NaN,NaN,NaN
4,20190508,558,558,542,546,196176,551.0,NaN,NaN,NaN,NaN,NaN
5,20190509,542,548,526,528,409774,546.6,NaN,NaN,NaN,NaN,NaN
6,20190510,518,539,518,538,302430,545.4,NaN,NaN,NaN,NaN,NaN
7,20190513,536,564,532,543,798705,542.6,NaN,NaN,NaN,NaN,NaN
8,20190514,541,552,527,551,300171,541.2,NaN,NaN,NaN,NaN,NaN
9,20190515,551,565,545,559,359249,543.8,547.4,NaN,NaN,NaN,NaN


In [53]:
ratio_list = [y/x-1 for x, y in zip(daily_data['close'].iloc[:-1], daily_data['close'].iloc[1:])]

In [58]:
ratio_list.index(max(ratio_list))

55

In [62]:
result = daily_data['close']*daily_data['volume']
result[55]

459027500

In [36]:
daily_data['close']*daily_data['volume']

0      380226840
1      249069450
2      568419140
3      181867700
4      185350140
5      395981680
6      322645400
7      283533440
8      504837160
9      329380260
10     480204600
11     451786140
12    2144151350
13     829118940
14     293951960
15     519756960
16     136600940
17     269515760
18     288727800
19     395991540
20     405833400
21    1277521200
22     617422390
23    1232785260
24     546348790
25    1344377420
26     347870600
27     737533440
28    1313097600
29    1676701840
         ...    
70     926188380
71      65892040
72     553786120
73    2211735400
74     976180480
75    4189515280
76     647162520
77     998643100
78    1351711200
79     612296880
80     273543760
81    3524466870
82    2120606060
83    1279946060
84     906806090
85     752374170
86    1152764860
87     340981900
88     626038400
89    1512872000
90     463520000
91     109016000
92    3122174800
93     555275480
94    1896997680
95     266446600
96     929002600
97     2451341

In [30]:
a = [i >= 0.15 for i in ratio_list]

In [32]:
len(a)

99

In [34]:
np.array(a)

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False])

In [35]:
for i,v in enumerate(a):
    print(i,v)

0 False
1 False
2 False
3 False
4 False
5 False
6 False
7 False
8 False
9 False
10 False
11 False
12 False
13 False
14 False
15 False
16 False
17 False
18 False
19 False
20 False
21 False
22 False
23 False
24 False
25 False
26 False
27 False
28 False
29 False
30 False
31 False
32 False
33 False
34 False
35 False
36 False
37 False
38 False
39 False
40 False
41 False
42 False
43 False
44 False
45 False
46 False
47 False
48 False
49 False
50 False
51 False
52 False
53 False
54 False
55 False
56 False
57 False
58 False
59 False
60 False
61 False
62 False
63 False
64 False
65 False
66 False
67 False
68 False
69 False
70 False
71 False
72 False
73 False
74 False
75 False
76 False
77 False
78 False
79 False
80 False
81 False
82 False
83 False
84 False
85 False
86 False
87 False
88 False
89 False
90 False
91 False
92 False
93 False
94 False
95 False
96 False
97 False
98 False


In [38]:
average_volume_test = [0 for i in range(99)]

In [48]:
ratio_test = np.array([0,1,1])
total_volume_test = np.array([True,False,True])
average_volume_test = np.array([0,0,1])
result = ratio_test * total_volume_test * average_volume_test

In [49]:
result

array([0, 0, 1])

In [66]:
a = strictly_increasing(np.array(daily_data), limit = 0.041, min_money_high_volume = 459027500)

In [67]:
a

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [65]:
# L: numpy array date, open, high, low, close, volume
#급등주 시점 찾아보자

def strictly_increasing(L, limit = 0.15, min_money_high_volume = 30000000000):
    #1) 종가비율 limit 이상
    ratio_list = [y/x-1 for x, y in zip(L[:,4][:-1], L[:,4][1:])]
    ratio_test = np.array([i >= limit for i in ratio_list])
    
    #2) 해당날 거래대금 300억 이상 거래대금 = 종가 * 거래량
    total_volume_list = L[:,4] * L[:,5]
    total_volume_test = [i >= min_money_high_volume  for i in total_volume_list]
    total_volume_test = np.array(total_volume_test[:-1])
    
    #3) 급등시점 전 평균 20일간 거래량의 1000%
    average_volume_list = ratio_test * total_volume_test
    average_volume_list_index = [ i for i,v in enumerate(average_volume_list) if v == True]
    
    average_volume_test = np.array([0 for i in range(99)])
    
    for ind in average_volume_list_index:
        if ind < 20:
            average_vol =  L[:,5][:ind].sum() / ind
        else:
            average_vol =  L[:,5][ind-20:ind].sum() / 20
        if L[:,5][ind] >= average_vol * 10:
            average_volume_test[ind] = 1

    
    return ratio_test * total_volume_test * average_volume_test

In [ ]:
# L: numpy array date, open, high, low, close, volume

def strictly_increasing(L, limit = 0.15, min_money_high_volume = 30000000000):
    #1) 종가비율 limit 이상
    ratio_list = [y/x-1 for x, y in zip(L[:,4][:-1], L[:,4][1:])]
    ratio_test = [i >= limit for i in ratio_list]
    
    #2) 해당날 거래대금 300억 이상
    total_volume_list = L[:,4] * L[:,5]
    volume_test = [i >= min_money_high_volume  for i in total_volume_list]
    volume_test = volume_test[:-1]
    
    
    
    
    #3) 급등시점 전 평균 ?일간 거래량의 1000%
    
    ind = L[:,5].argmax()
    high_vol = L[:,5][ind]
    if ind >= 20:
        average_vol =  L[:,5][ind-20:ind].sum() / 20
    else:
        average_vol =  L[:,5][:ind].sum() / ind
    
    all([high_vol >= average_vol * 10,
         high_vol * L[:,4][ind] >= min_money_high_volume])
    
    #3) 첫번째급등 이후 급등하는 곳 찾기 (for 급등의 조건확인)
    target_ind = np.array(ratio_list[ind:]).argmax()
    target_ratio_inde
    
    
    sum_vol20 = 0
    today_vol = 0
    #index와 값을 같이 조합
    for i, vol in enumerate(L[:,5][:ind]):
        if i == 0:
            today_vol = vol
        elif 1 <= i <= 20:
            sum_vol20 += vol
        else:
            break
    
    volume_ratio_list = max


    
    return 

In [ ]:
#!!주, 월 increasing 체크 로직 만들 것.


In [39]:
def strictly_increasing(L):
    return all(x<y for x, y in zip(L, L[1:]))

def strictly_decreasing(L):
    return all(x>y for x, y in zip(L, L[1:]))

def non_increasing(L):
    return all(x>=y for x, y in zip(L, L[1:]))

def non_decreasing(L):
    return all(x<=y for x, y in zip(L, L[1:]))

def monotonic(L):
    return non_increasing(L) or non_decreasing(L)

In [ ]:
def check_speedy_rising_volume(code):
    today = datetime.datetime.today().strftime("%Y%m%d")
    volumes = df['volume']

    if len(volumes) < 21:
        return False

    sum_vol20 = 0
    today_vol = 0
    #index와 값을 같이 조합
    for i, vol in enumerate(volumes):
        if i == 0:
            today_vol = vol
        elif 1 <= i <= 20:
            sum_vol20 += vol
        else:
            break

    avg_vol20 = sum_vol20 / 20
    if today_vol > avg_vol20 * 10:
        return True

In [ ]:
def check_speedy_rising_volume(code):
    today = datetime.datetime.today().strftime("%Y%m%d")
    volumes = df['volume']

    if len(volumes) < 21:
        return False

    sum_vol20 = 0
    today_vol = 0

    for i, vol in enumerate(volumes):
        if i == 0:
            today_vol = vol
        elif 1 <= i <= 20:
            sum_vol20 += vol
        else:
            break

    avg_vol20 = sum_vol20 / 20
    if today_vol > avg_vol20 * 10:
        return True

In [ ]:
def update_buy_list(self, buy_list):
    f = open("buy_list.txt", "wt")
    for code in buy_list:
        f.writelines("매수;", code, ";시장가;10;0;매수전")
    f.close()

In [ ]:
def run(self):
    buy_list = []
    num = len(self.kosdaq_codes)

    for i, code in enumerate(self.kosdaq_codes):
        print(i, '/', num)
        if self.check_speedy_rising_volume(code):
            buy_list.append(code)

    self.update_buy_list(buy_list)

In [ ]:
if __name__ == "__main__":
    app = QApplication(sys.argv)
    pymon = PyMon()
    pymon.run()